# Computing the electronic ground state energy of a hydrogen molecule

## Setup

First, get an API key from IonQ. This will be used by the IonQ provider inside Qiskit to submit circuits to the IonQ platform.

After securing an API key, install the qiskit-ionq provider at https://github.com/Qiskit-Partners/qiskit-ionq/

### (Optional) Extra Dependencies

Some examples use additional Python dependencies; please make sure to `pip install` them as needed.

Dependencies:
* `matplotlib`: To run `qiskit.visualization.plot_histogram`.

**NOTE**: The provider expects an API key to be supplied via the `token` keyword argument to its constructor. If no token is directly provided, the provider will check for one in the `QISKIT_IONQ_API_TOKEN` environment variable.

Now that the Python package has been installed, you can import and instantiate the provider:

In [1]:
#import Aer here, before calling qiskit_ionq_provider
from qiskit import Aer

from qiskit_ionq_provider import IonQProvider 

#Call provider and set token value
provider = IonQProvider(token='My token')

The `provider` instance can now be used to create and submit circuits to IonQ.

### Backend Types

The IonQ provider supports two backend types:
* `ionq_simulator`: IonQ's simulator backend.
* `ionq_qpu`: IonQ's QPU backend.

To view all current backend types, use the `.backends` property on the provider instance:


In [2]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7f9fcd7e7208>()>,
 <IonQQPUBackend('ionq_qpu') from <qiskit_ionq_provider.ionq_provider.IonQProvider object at 0x7f9fcd7e7208>()>]

### Let's create a guess ground state

We try $e^{\theta(T-T^\dagger)}$, where $\theta$ is our variational parameter and $T = a_3^\dagger a_2^\dagger a_1 a_0$ is our excitation operator. Qubits 0 and 1 denote our Hartree-Fock ground state (the good initial guess), and we are aiming to explore its vicinity by exciting a pair of electrons to the next level up, which are encoded in qubits 2 and 3. The two $NOT$ gates applied to qubits 0 and 1 at the very beginning (see below) correspond to loading of the two electrons in total to the levels commensurate with the good initial guess.

In [3]:
from qiskit import QuantumCircuit

def load_qasm(fname):
    # load circuit template from qasm file
    circ = QuantumCircuit.from_qasm_file(fname)
    return circ

# Create the ansatz state
qc = load_qasm('ZZZZ.qasm')

# Show the circuit:
qc.draw()

┌───┐┌───┐                                                       ┌───┐»
q_0: ┤ X ├┤ H ├────────────■───────────────────────────────────────■──┤ H ├»
     ├───┤├───┤            │                                ┌───┐  │  ├───┤»
q_1: ┤ X ├┤ H ├────────────┼────■────────────────────────■──┤ H ├──┼──┤ H ├»
     ├───┤└───┘            │    │                        │  ├───┤  │  ├───┤»
q_2: ┤ H ├─────────────────┼────┼────■──────────────■────┼──┤ H ├──┼──┤ S ├»
     ├───┤┌───┐┌───┐┌───┐┌─┴─┐┌─┴─┐┌─┴─┐┌────────┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐├───┤»
q_3: ┤ S ├┤ S ├┤ S ├┤ H ├┤ X ├┤ X ├┤ X ├┤ RZ(-1) ├┤ X ├┤ X ├─────┤ X ├┤ H ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└────────┘└───┘└───┘     └───┘└───┘»
«     ┌───┐                                                       ┌───┐┌───┐»
«q_0: ┤ H ├────────────■───────────────────────────────────────■──┤ H ├┤ S ├»
«     └───┘            │                                ┌───┐  │  ├───┤└───┘»
«q_1: ─────────────────┼────■────────────────────────■──┤ H ├──┼──┤ H ├─────»
«     ┌───┐┌───┐┌───┐  │    │                        │  ├───┤  │  ├───┤┌───┐»
«q_2: ┤ S ├┤ S ├┤ H ├──┼────┼────■──────────────■────┼──┤ H ├──┼──┤ S ├┤ S ├»
«     ├───┤├───┤└───┘┌─┴─┐┌─┴─┐┌─┴─┐┌────────┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐├───┤├───┤»
«q_3: ┤ S ├┤ H ├─────┤ X ├┤ X ├┤ X ├┤ RZ(-1) ├┤ X ├┤ X ├─────┤ X ├┤ H ├┤ S ├»
«     └───┘└───┘     └───┘└───┘└───┘└────────┘└───┘└───┘     └───┘└───┘└───┘»
«     ┌───┐┌───┐┌───┐                                             ┌───┐┌───┐»
«q_0: ┤ S ├┤ S ├┤ H ├──■───────────────────────────────────────■──┤ H ├┤ S ├»
«     └───┘└───┘└───┘  │                                ┌───┐  │  ├───┤└───┘»
«q_1: ─────────────────┼────■────────────────────────■──┤ H ├──┼──┤ H ├─────»
«     ┌───┐┌───┐┌───┐  │    │                        │  ├───┤  │  ├───┤┌───┐»
«q_2: ┤ S ├┤ S ├┤ H ├──┼────┼────■──────────────■────┼──┤ H ├──┼──┤ S ├┤ H ├»
«     ├───┤├───┤├───┤┌─┴─┐┌─┴─┐┌─┴─┐┌────────┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐├───┤├───┤»
«q_3: ┤ S ├┤ S ├┤ H ├┤ X ├┤ X ├┤ X ├┤ RZ(-1) ├┤ X ├┤ X ├─────┤ X ├┤ H ├┤ S ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└────────┘└───┘└───┘     └───┘└───┘└───┘»
«     ┌───┐┌───┐┌───┐┌───┐                                            ┌───┐»
«q_0: ┤ S ├┤ S ├┤ S ├┤ H ├──■──────────────────────────────────────■──┤ H ├»
«     └───┘└───┘└───┘└───┘  │                               ┌───┐  │  ├───┤»
«q_1: ──────────────────────┼────■───────────────────────■──┤ H ├──┼──┤ S ├»
«                           │    │                       │  ├───┤  │  ├───┤»
«q_2: ──────────────────────┼────┼────■─────────────■────┼──┤ H ├──┼──┤ H ├»
«     ┌───┐               ┌─┴─┐┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐├───┤»
«q_3: ┤ H ├───────────────┤ X ├┤ X ├┤ X ├┤ RZ(1) ├┤ X ├┤ X ├─────┤ X ├┤ H ├»
«     └───┘               └───┘└───┘└───┘└───────┘└───┘└───┘     └───┘└───┘»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐                                            »
«q_0: ┤ S ├┤ S ├┤ S ├┤ S ├┤ H ├──■──────────────────────────────────────■──»
«     ├───┤├───┤├───┤└───┘└───┘  │                               ┌───┐  │  »
«q_1: ┤ S ├┤ S ├┤ H ├────────────┼────■───────────────────────■──┤ H ├──┼──»
«     └───┘└───┘└───┘            │    │                       │  ├───┤  │  »
«q_2: ───────────────────────────┼────┼────■─────────────■────┼──┤ H ├──┼──»
«     ┌───┐┌───┐┌───┐┌───┐     ┌─┴─┐┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐»
«q_3: ┤ S ├┤ S ├┤ S ├┤ H ├─────┤ X ├┤ X ├┤ X ├┤ RZ(1) ├┤ X ├┤ X ├─────┤ X ├»
«     └───┘└───┘└───┘└───┘     └───┘└───┘└───┘└───────┘└───┘└───┘     └───┘»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐                                       »
«q_0: ┤ H ├┤ S ├┤ S ├┤ S ├┤ S ├┤ H ├──■────────────────────────────────────»
«     ├───┤├───┤├───┤├───┤├───┤└───┘  │                               ┌───┐»
«q_1: ┤ S ├┤ S ├┤ S ├┤ S ├┤ H ├───────┼────■───────────────────────■──┤ H ├»
«     ├───┤├───┤├───┤├───┤└───┘       │    │                       │  ├───┤»
«q_2: ┤ S ├┤ S ├┤ S ├┤ H ├────────────┼────┼────■─────────────■────┼──┤ H ├»
«     ├───┤├───┤├───┤└───┘          ┌─┴─┐┌─┴─┐┌─┴─┐┌───────┐

Once we can generate a guess state, we need to append the circuit that generates the state by a suite of suitable basis transformation circuits that correspond to each term of the Hamiltonian, energy operator. To do so, we start by considering a sample configuration of a hydrogen molecule with the inter-core spacing of 0.712234A. We obtain (by running any choice of your favorite computational chemistry programs, such as psi4 -- we skip this step herein, as it is beyond the scope of this demo)

$H_{\rm MO,electron} = -1.2703 (a_0^\dagger a_0 + a_1^\dagger a_1) -0.4586 (a_2^\dagger a_2 + a_3^\dagger a_3) - 0.6801 (a_1^\dagger a_0^\dagger a_0 a_1) - 0.1797 (a_1^\dagger a_0^\dagger a_2 a_3 + a_3^\dagger a_2^\dagger a_0 a_1) - 0.4889 (a_2^\dagger a_0^\dagger a_0 a_2 + a_3^\dagger a_1^\dagger a_1 a_3) -0.6686 (a_2^\dagger a_1^\dagger a_1 a_2 + a_3^\dagger a_0^\dagger a_0 a_3) +0.1797 (a_2^\dagger a_1^\dagger a_0 a_3) - 0.7028 (a_3^\dagger a_2^\dagger a_2 a_3),$

where the subindices label the different molecular orbitals (MOs) and the included terms of length two contain nucleus-electron interaction and electron kinetic energy and the included terms of length four contain electron-electron interaction. The nucleus-nucleus interaction is computed classically and separately, which amounts in our example case to 0.7430 (Ha). Thus, the expectation value of the total energy is computed according to $\langle H_{\rm MO,electron} \rangle + 0.7430$ (Ha), where $\langle .. \rangle$ denotes an expectation value associated with a prepared ansatz state (state-dependent notations omitted for brevity). 

To now evaluate $\langle H_{\rm MO,electron} \rangle + 0.7430$ on a quantum computer, we apply the JW transformation introduced in the slides earlier. We obtain

$H_{\rm MO, electron, JW} + 0.7430 = -0.0597 I_0 I_1 I_2 I_3 -0.0449 X_0 X_1 Y_2 Y_3 + 0.0449 X_0 Y_1 Y_2 X_3 + 0.0449 Y_0 X_1 X_2 Y_3 - 0.0449 Y_0 Y_1 X_2 X_3 + 0.1758 Z_0 I_1 I_2 I_3 + 0.1700 Z_0 Z_1 I_2 I_3 + 0.1222 Z_0 I_1 Z_2 I_3 + 0.1671 Z_0 I_1 I_2 Z_3 + 0.1756 I_0 Z_1 I_2 I_3 + 0.1671 I_0 Z_1 Z_2 I_3 + 0.1222 I_0 Z_1 I_2 Z_3 -0.2367 I_0 I_1 Z_2 I_3 + 0.1757 I_0 I_1 Z_2 Z_3 -0.2367 I_0 I_1 I_2 Z_3.$

Each term is of the form of a product of pauli matrices $X,Y,Z$. As discussed in the slides, to evalualte an expectation value of $X_k,Y_k,Z_k$, applied to say a $k$th qubit, we append $H, S^{\dagger}H, I$ gate(s) to the $k$th qubit. For a product of pauli matrices, applied to multiple qubits, we append corresponding gates to the multiple qubits appropriately. For instance, to compute the expectation value of $X_0 X_1 X_2 X_3$, applied to qubits 0 through 3, we append $H$ gates to qubits 0 through 3. All circuits required for each term shown in $H_{\rm MO, electron, JW}$ are included as QASM files. Note most of the terms admit the same circuit appendix, namely, the pauli strings with either $I$ or $Z$ as its only multiplicands. Further note the very first term with the string $I_0 I_1 I_2 I_3$ does not require an evaluation by the quantum computer, as its expectation value is always 1.

We are now ready to run the circuits. We start with the simulator.

In [4]:
def cpar(c,a,nn):
    # parametrize the circuit
    n = QuantumCircuit(nn)
    for g in c:
        if g[0].name=='rz':
            #print(g[0].params[0])
            n.rz(g[0].params[0]*a/2**(nn-1),g[1][0].index)
        else:
            n.append(g[0],[q.index for q in g[1]])
    return n

def zbin(x,ss,nn):
    mm = [-(((x&(2**q))//2**q)*2-1) for q in range(2**nn)]
    r = 1
    for s in ss:
        r = r*mm[s]
    return r

def get_ham4(ress): 
    ham  = -0.0597
    ham -=  0.0449*sum([zbin(x,[0,1,2,3],4)*ress[1][x] for x in range(16)])
    ham +=  0.0449*sum([zbin(x,[0,1,2,3],4)*ress[2][x] for x in range(16)])
    ham +=  0.0449*sum([zbin(x,[0,1,2,3],4)*ress[3][x] for x in range(16)])
    ham -=  0.0449*sum([zbin(x,[0,1,2,3],4)*ress[4][x] for x in range(16)])
    ham +=  0.1758*sum([zbin(x,[0],4)*ress[0][x] for x in range(16)])
    ham +=  0.1700*sum([zbin(x,[0,1],4)*ress[0][x] for x in range(16)])
    ham +=  0.1222*sum([zbin(x,[0,2],4)*ress[0][x] for x in range(16)])
    ham +=  0.1671*sum([zbin(x,[0,3],4)*ress[0][x] for x in range(16)])
    ham +=  0.1756*sum([zbin(x,[1],4)*ress[0][x] for x in range(16)])
    ham +=  0.1671*sum([zbin(x,[1,2],4)*ress[0][x] for x in range(16)])
    ham +=  0.1222*sum([zbin(x,[1,3],4)*ress[0][x] for x in range(16)])
    ham -=  0.2367*sum([zbin(x,[2],4)*ress[0][x] for x in range(16)])
    ham +=  0.1757*sum([zbin(x,[2,3],4)*ress[0][x] for x in range(16)])
    ham -=  0.2367*sum([zbin(x,[3],4)*ress[0][x] for x in range(16)])
    
    #for i in range(5):
    #    print(ress[i])
    
    return ham

def get_pops(res,nn,n):
    #print(res)
    pops = [0 for i in range(2**nn)]
    for key in res.keys():
        pops[int(key,2)] = res[key]/n
    return pops
    
fqsm4 = ['ZZZZ.qasm','XXYY.qasm','XYYX.qasm','YXXY.qasm','YYXX.qasm']

In [6]:
from qiskit.providers.jobstatus import JobStatus
from qiskit import Aer, execute
from qiskit import ClassicalRegister

# Import parametrizable circuits
circs4 = []
for fname in fqsm4:
    circs4.append(load_qasm(fname))

# Set the parameter $\theta$
#theta = 0.2144802815837562
theta = 0.0

# Choose the simulator backend 
backend = provider.get_backend("ionq_simulator")

# Run the circuit:
def run_jobs(backend,circs,theta,nn,nshots):
    jobs = []
    job_ids = []
    qcs = []
    cr = ClassicalRegister(nn,'c')

    for circ in circs:
        qc = cpar(circ,theta,nn)
        qc.add_register(cr)
        qc.measure(range(nn),range(nn))
        #print(qc.draw())
        qcs.append(qc)
        job = backend.run(qc, shots=nshots)
        jobs.append(job)
        job_ids.append(job.job_id())
    
    return jobs

jobs4 = run_jobs(backend,circs4,theta,4,1000)

# Fetch the result
def get_jobs(jobs,nn,nshots):
    results = []
    for i in range(len(jobs)):
        result = jobs[i].result()
        results.append(get_pops(result.data()['counts'],nn,nshots))
    
    return results

results4 = get_jobs(jobs4,4,1000)

# Compute the expectation value $\langle H_{\rm MO,electron} \rangle + 0.7430 = \langle H_{\rm MO,electron,JW} \rangle$.
ham4 = get_ham4(results4)

# Print the total energy $\langle H_{\rm MO,electron,JW} \rangle$ (Ha).
print(ham4)

-1.1174


We have now computed an expectation value of the total energy of a hydrogen molecule with the inter-core spacing of 0.712234A. Specifically, we computed the expectation value for a prepared guess state that corresponds to $\theta = 0$, which coincides with the Hartree-Fock ground state, i.e., the good initial guess state. To find the ground state, which is our goal here, we rely on the following strategy. Note our quantum computer efficiently calculates the expectation value of the energy of a prepared guess state. A simple method to take advantage of this may be to consider the well-known Nelder-Mead method. Roughly speaking, this iterative method keeps track of, for an $N$ variable optimization problem, $N+1$ sets of $N$ variables, while updating the elements of the set(s) one iteration at a time, based on the values of the optimization function for the $N+1$ sets of $N$ variables at a given iteration. With an appropriate convergence criterion, if the problem admits a convergence, the method converges to a local minimum. The location of the local minimum that the method converges to depends in general on the initial choice of the $N+1$ sets of $N$ variables.

Below, we explicitly work out an example of this. Our initial choice of angles $\theta$ are in principle arbitrary. Note small $\theta$ values would be a good choice should our initial good guess (Hartree-Fock) is indeed ``good''.

In [7]:
from scipy.optimize import minimize

# Nelder-Mead implementation
def obj4(backend,circs,theta):
    print(theta)
    jobs = run_jobs(backend,circs,theta,4,100000)
    results = get_jobs(jobs,4,100000)
    return get_ham4(results)

# Show the convergence?
result = minimize(lambda x: obj4(backend,circs4,x[0]),0.0,method='nelder-mead',tol=0.0001)
print(result)
theta_opt = result.x[0]
print(obj4(backend,circs4,theta_opt))

0.0
0.00025
0.0005
0.00075
0.00125
0.0017500000000000003
0.0027500000000000007
0.003750000000000001
0.005750000000000002
0.0077500000000000025
0.011750000000000003
0.015750000000000004
0.023750000000000004
0.03175000000000001
0.047750000000000015
0.06375000000000003
0.09575000000000006
0.12775000000000009
0.19175000000000014
0.2557500000000002
0.2557500000000002
0.2237500000000002
0.25575000000000025
0.20775000000000016
0.19175000000000011
0.21575000000000016
0.22375000000000017
0.21175000000000016
0.20775000000000016
0.21375000000000016
0.20975000000000016
0.21275000000000016
0.21375000000000016
0.21225000000000016
0.21325000000000016
0.21250000000000016
0.21250000000000016
0.21300000000000016
0.21287500000000015
0.21262500000000017
0.21287500000000015
0.21268750000000017
0.21268750000000017
 final_simplex: (array([[0.21275  ],
       [0.2126875]]), array([-1.13674053, -1.13671179]))
           fun: -1.136740528
       message: 'Optimization terminated successfully.'
          nfev: 4

To do this on a quantum computer, we slightly modify the commands as follows. We evaluate the energy expectation values at both the good initial guess and the converged points (see above for the convergence obtained via the simulator). Note we have optimized our framework at various levels, as today's quantum computer has relatively large noise.

In [30]:
fqsm2 = ['ZZ.qasm','XX.qasm','YY.qasm']

def get_ham2(ress):
    ham  =  0.28604714
    ham -=  0.47331*sum([zbin(x,[1],2)*ress[0][x] for x in range(1,3)])/sum([ress[0][x] for x in range(1,3)])
    ham +=  0.35151*sum([zbin(x,[0],2)*ress[0][x] for x in range(1,3)])/sum([ress[0][x] for x in range(1,3)])
    ham -=  0.57874
    ham +=  0.08984*sum([zbin(x,[0,1],2)*ress[1][x] for x in range(4)])
    ham +=  0.08984*sum([zbin(x,[0,1],2)*ress[2][x] for x in range(4)])
    
    for i in range(3):
        print(ress[i])
    
    return ham

# Import parametrizable circuits
circs = []
for fname in fqsm2:
    circs.append(load_qasm(fname))

# Switch the backend to run circuits on a quantum computer
qpu_backend = provider.get_backend("ionq_qpu")

jobs_zero = run_jobs(qpu_backend,circs,0.0,2,1000)
jobs_opt = run_jobs(qpu_backend,circs,theta_opt,2,1000)

The job will queue, and results will arrive once it's executed!

In [31]:
#Check if jobs are done
for i in range(len(jobs_zero)):
    print(jobs_zero[i].status())

for i in range(len(jobs_opt)):
    print(jobs_opt[i].status())

JobStatus.QUEUED
JobStatus.QUEUED
JobStatus.QUEUED
JobStatus.QUEUED
JobStatus.QUEUED
JobStatus.QUEUED


In [29]:
# Fetch the result
results_zero = get_jobs(jobs_zero,2,1000)
results_opt = get_jobs(jobs_opt,2,1000)

# Compute the expectation value $\langle H_{\rm MO,electron} \rangle + 0.7430 = \langle H_{\rm MO,electron,JW} \rangle$.
ham2_zero = get_ham2(results_zero)
ham2_opt = get_ham2(results_opt)

# Print the total energy $\langle H_{\rm MO,electron,JW} \rangle$ (Ha).
print(ham2_zero,ham2_opt)

[0.028, 0.97, 0, 0.002]
[0.282, 0.207, 0.287, 0.224]
[0.289, 0.269, 0.231, 0.211]
[0.03, 0.943, 0.025, 0.002]
[0.155, 0.348, 0.269, 0.228]
[0.263, 0.251, 0.296, 0.19]
-1.11643478 -1.104376041157025
